<a href="https://colab.research.google.com/github/pk2971/Capstone/blob/main/Code/Text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset  # Import the Dataset class
from tqdm import tqdm

In [61]:
import re
keywords_to_match = ['she', 'her', 'hers', 'lady', 'ladies', 'woman', 'women', 'girl', 'girls']

def extract_sentences_with_keywords(text, keywords):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    filtered_sentences = [
        sentence for sentence in sentences
        if any(re.search(rf'\b{re.escape(keyword)}\b', sentence, re.IGNORECASE) for keyword in keywords)
    ]

    return '\n'.join(filtered_sentences)


In [63]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"  # You can choose a different BART variant
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


file_path = '/content/1927.txt'
# Example text
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
filtered_sentences=extract_sentences_with_keywords(content, keywords_to_match)

input_ids = tokenizer.encode(filtered_sentences, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(input_ids, max_length=10000, num_beams=10, length_penalty=2.5,num_return_sequences=3, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:", summary)

Generated Summary: Lords absence noble friend Lord Peel beg move Motion stands name upon Paper Lords Lord Chancellor. Lord Harris moved Motion refer Bill Select Committee still deeply impressed arguments going repeat arguments length gist first place noble Lord brought forward example hasty legislation popularly known as People's Budget 1909 Lordships well aware agriculture country flourishing condition might one reasons Lord Harris adduced fact operation Budget.


In [69]:
len(filtered_sentences)/len(content)

0.4184785268210573

In [65]:
clean_summary = summary.strip()


In [66]:
print(clean_summary)

Lords absence noble friend Lord Peel beg move Motion stands name upon Paper Lords Lord Chancellor. Lord Harris moved Motion refer Bill Select Committee still deeply impressed arguments going repeat arguments length gist first place noble Lord brought forward example hasty legislation popularly known as People's Budget 1909 Lordships well aware agriculture country flourishing condition might one reasons Lord Harris adduced fact operation Budget.


In [64]:
with open("/content/output_summary_before_suffrage_year(woman).txt", "w", encoding="utf-8") as output_file:
    output_file.write(clean_summary)

print("Summary saved to output_summary.txt")

Summary saved to output_summary.txt
